In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
import tensorflow
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess

from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import preprocess_input as vgg_preprocess

from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.models import Sequential,load_model

In [ ]:
with np.load('/content/gdrive/MyDrive/sp_train.npz') as data:
    X_train = data["dataset"]
    y_train = data["labels"]

with np.load('/content/gdrive/MyDrive/sp_valid.npz') as data:
    X_valid = data["dataset"]
    y_valid = data["labels"]

In [ ]:
with np.load('/content/gdrive/MyDrive/sp_test.npz') as data:
    dataset = data["dataset"]
    labels = data["labels"]

In [ ]:
y_train = tensorflow.keras.utils.to_categorical(y_train, 2)
y_valid = tensorflow.keras.utils.to_categorical(y_valid, 2)

In [ ]:
input_shape = (224, 224, 3)
input = dataset[:1]

base_model = ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False
output = base_model(input)
print(output.shape)

94773248/94765736 [==============================] - 1s 0us/step
(1, 7, 7, 2048)


In [ ]:
global_average_layer = keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(output)
print(feature_batch_average.shape)
prediction_layer = keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(1, 2048)
(1, 2)


In [ ]:
inputs = keras.Input(shape=input_shape)
x = resnet_preprocess(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

In [ ]:
def fn(correct, predicted,train_temp=100): 
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                    logits=predicted/train_temp)

In [ ]:
optimizer=keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer,
              loss=fn,
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098  

In [ ]:
history = model.fit(X_train, soft, validation_data=(X_valid, y_valid), batch_size=16, epochs=15)

Epoch 1/15
962/962 [==============================] - 145s 147ms/step - loss: 0.6931 - accuracy: 0.7088 - val_loss: 0.6923 - val_accuracy: 0.5409
Epoch 2/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6930 - accuracy: 0.8474 - val_loss: 0.6916 - val_accuracy: 0.5492
Epoch 3/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6930 - accuracy: 0.8837 - val_loss: 0.6911 - val_accuracy: 0.5546
Epoch 4/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6930 - accuracy: 0.8984 - val_loss: 0.6906 - val_accuracy: 0.5586
Epoch 5/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6929 - accuracy: 0.9090 - val_loss: 0.6902 - val_accuracy: 0.5608
Epoch 6/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6929 - accuracy: 0.9183 - val_loss: 0.6899 - val_accuracy: 0.5677
Epoch 7/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6929 - accuracy: 0.9272 - val_loss: 0.6896 -

In [ ]:
model.save('/content/gdrive/My Drive/models/sp_model_studentv1.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
classes=model.predict(X_train)

In [ ]:
soft=tf.nn.softmax(classes/100)

In [ ]:
soft

<tf.Tensor: shape=(15390, 2), dtype=float32, numpy=
array([[0.5072666 , 0.49273342],
       [0.51622945, 0.48377058],
       [0.49936235, 0.50063765],
       ...,
       [0.4897501 , 0.5102499 ],
       [0.50214803, 0.49785194],
       [0.5033197 , 0.49668026]], dtype=float32)>

In [ ]:
print(len(base_model.layers))
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 172

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

175


In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0001),
              loss=fn,
              metrics=['accuracy', 'AUC'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=16, epochs=15)

Epoch 1/15
962/962 [==============================] - 147s 148ms/step - loss: 0.6936 - accuracy: 0.5152 - val_loss: 0.6969 - val_accuracy: 0.5449
Epoch 2/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6936 - accuracy: 0.5229 - val_loss: 0.6969 - val_accuracy: 0.5417
Epoch 3/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6936 - accuracy: 0.5192 - val_loss: 0.6969 - val_accuracy: 0.5403
Epoch 4/15
962/962 [==============================] - 139s 144ms/step - loss: 0.6936 - accuracy: 0.5199 - val_loss: 0.6969 - val_accuracy: 0.5395
Epoch 5/15
962/962 [==============================] - 139s 145ms/step - loss: 0.6936 - accuracy: 0.5160 - val_loss: 0.6969 - val_accuracy: 0.5411
Epoch 6/15
962/962 [==============================] - 140s 145ms/step - loss: 0.6936 - accuracy: 0.5208 - val_loss: 0.6969 - val_accuracy: 0.5395
Epoch 7/15
962/962 [==============================] - 139s 145ms/step - loss: 0.6936 - accuracy: 0.5174 - val_loss: 0.6969 -

In [ ]:
preds = np.argmax(tensorflow.nn.softmax(model.predict(dataset), axis=-1))

In [ ]:
model = tensorflow.keras.models.load_model('/content/gdrive/MyDrive/models/sp_model_15_softmax2.h5')

In [ ]:
with np.load('/content/gdrive/MyDrive/sp_adv_train_2.npz') as data:
    dataset1 = data["dataset"]
    labels1 = data["labels"]

In [ ]:
correct = []
for i in range(len(dataset)):
    correct.append(np.argmax(tensorflow.nn.softmax(model1.predict(dataset[i:i+1]), axis=-1)) == labels[i])

Counter(correct)

Counter({False: 132, True: 268})

In [ ]:
np.argmax(tensorflow.nn.softmax(model.predict(dataset), axis=-1))

0